In [1]:
import requests
from bs4 import BeautifulSoup
import time
import os
import openai

folder_path = "Unsheathed"
# Create the folder if it doesn't exist
os.makedirs(folder_path, exist_ok=True)

api_key = os.environ.get('OPENAI_APIKEY')
openai.api_key = api_key

GPT_MODEL = "gpt-3.5-turbo"

conversation = [
    {"role" : "system", "content" : "Translate this from Chinese to English. The context being that this is a Xianxia novel. Do not write anything else"},
  ]

# conversation_nopop = [
#     {"role" : "system", "content" : "Translate this from Chinese to English. The context being that this is a Xianxia novel. Do not write anything else"},
#   ]

In [2]:
def GPTranslate(para, totaltokens):
    conversation.append({"role": "user", "content": para})
        
    response = openai.ChatCompletion.create(
    model=GPT_MODEL,
    messages= conversation
    )

    conversation.pop()

    # print("for para of length", len(para), "\n", response['usage'])
    totaltokens += response['usage']['total_tokens']
    # print(response['choices'][0]['message']['content'])
    
    return response['choices'][0]['message']['content'], totaltokens

In [3]:
url = 'https://lnmtl.com/chapter/come-sword-book-1-chapter-1'
Chapters = 1

In [4]:
for i in range(3):
    
    print("at chapter", url)
    
    # time.sleep(3)
    response = requests.get(url)

    paras = []

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the HTML content of the page using BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find all elements with the specified class 'original'
        nextchap = soup.find('li', class_='next').find('a')['href']
        # print(nextchap)
        url = nextchap
        
        # Find all elements with the specified class 'original'
        original_sentences = soup.find_all('sentence', class_='original')
        
        # Extract and print the text from each element
        for sentence in original_sentences:
            line = sentence.get_text()
            paras.append(line)
        
                
        # Initialize variables
        running_length = 0  # Start with 0
        threshold = 500  # Set your desired threshold here
        result = []  # Store the joined paragraphs
        totaltokens = 0 # Total tokens used in translating one chapter
        translation = [] # Storing the translation

        
        title, totaltokens = GPTranslate(soup.find('span', class_="chapter-title")['data-content'], totaltokens)
        title = "Chapter " + str(Chapters) + " - " + title.rstrip(".")
        print(title)
        Chapters += 1
        
        # Iterate over the paras
        for para in paras:
            # Check if adding the current paragraph exceeds the threshold
            if running_length + len(para) <= threshold:
                running_length += len(para) # Update the running length
                result.append(para)
                
            else:
                # print(''.join(result))
                # time.sleep(2)
                translated_result, totaltokens = GPTranslate(''.join(result), totaltokens)
                print(translated_result)
                translation.append(translated_result)
                
                # Reset the result and running_length
                result = [para]  # Start a new result with the current paragraph
                running_length = len(para)  # Set running_length to the current paragraph's length

        # print(''.join(result))
        # time.sleep(2)
        translated_result, totaltokens = GPTranslate(''.join(result), totaltokens)
        print(translated_result)
        translation.append(translated_result)
        
        tokensusedline = "Note: This translation used " + str(totaltokens) + " GPT tokens"
        translation.append(tokensusedline)
        print(tokensusedline)
        
        filename = f"{title}.txt"
        full_path = os.path.join(folder_path, filename)
        with open(full_path, 'w') as file:
            file.write(''.join(translation))
        print("saved to", full_path)
            
    else:
        print("Failed to retrieve the webpage. Status code:", response.status_code)

at chapter https://lnmtl.com/chapter/come-sword-book-1-chapter-1
Chapter 1 - Startled Awakening
On the second day of the second month, the dragon awakens.
In the twilight, in a quiet place called Mud Bottle Alley, there is a lonely and thin young man. At this moment, he is following the tradition, holding a candle in one hand and a peach branch in the other, shining on the beams, walls, wooden bed, and other places, using the peach branch to tap and drive away snakes, scorpions, centipedes, etc. He mutters words in his mouth, an old saying passed down through generations in this town: "On the second day of the second month, the candle illuminates the beams, the peach strikes the walls, nowhere for the snakes and insects to hide in this world."
The young man's surname is Chen, his given name is Ping'an, and his parents passed away early. The porcelain of this small town is highly renowned. Since the founding of the dynasty, it has taken up the important task of "burning sacrificial vess

In [6]:
# def GPTranslate_nopop(para):
#     conversation_nopop.append({"role": "user", "content": para})
        
#     response = openai.ChatCompletion.create(
#     model=GPT_MODEL,
#     messages= conversation_nopop
#     )
    
#     conversation_nopop.append({"role" : "assistant", "content": response['choices'][0]['message']['content']})

#     # conversation.pop()

#     # print("for para of length", len(para), "\n", response['usage'])
#     tokensused = response['usage']['total_tokens']
#     # print(response['choices'][0]['message']['content'])
    
#     return response['choices'][0]['message']['content'], tokensused

# url = 'https://lnmtl.com/chapter/come-sword-book-1-chapter-1'
# Chapters = 1
# for i in range(1):
#     conversation_nopop = conversation_nopop[:1]
#     print("at chapter", url)
    
#     # time.sleep(3)
#     response = requests.get(url)

#     paras = []

#     # Check if the request was successful (status code 200)
#     if response.status_code == 200:
#         # Parse the HTML content of the page using BeautifulSoup
#         soup = BeautifulSoup(response.text, 'html.parser')
        
#         # Find all elements with the specified class 'original'
#         nextchap = soup.find('li', class_='next').find('a')['href']
#         # print(nextchap)
#         url = nextchap
        
#         # Find all elements with the specified class 'original'
#         original_sentences = soup.find_all('sentence', class_='original')
        
#         # Extract and print the text from each element
#         for sentence in original_sentences:
#             line = sentence.get_text()
#             paras.append(line)
        
                
#         # Initialize variables
#         running_length = 0  # Start with 0
#         threshold = 500  # Set your desired threshold here
#         result = []  # Store the joined paragraphs
#         runningtokens = 0 # Total tokens used in current conversation
#         totaltokens = 0 # Total tokens used overall
#         translation = [] # Storing the translation

        
#         title, runningtokens = GPTranslate_nopop(soup.find('span', class_="chapter-title")['data-content'])
#         totaltokens += runningtokens
#         title = "NPChapter " + str(Chapters) + " - " + title.rstrip(".")
#         print(title)
#         Chapters += 1
        
#         # Iterate over the paras
#         for para in paras:
#             # Check if adding the current paragraph exceeds the threshold
#             if running_length + len(para) <= threshold:
#                 running_length += len(para) # Update the running length
#                 result.append(para)
                
#             else:
#                 print("entered else block")
#                 # print(''.join(result))
#                 # time.sleep(2)
#                 print("conversation no pop \n", conversation_nopop[-1])
#                 translated_result, runningtokens = GPTranslate_nopop(''.join(result))
#                 print("translation successful, with running", runningtokens)
#                 totaltokens += runningtokens

#                 # print(translated_result)
#                 translation.append(translated_result)
                
#                 if runningtokens >= 3800:
#                     print("pruning tokenusage")
#                     conversation_nopop = [conversation_nopop[0], conversation_nopop[-1]]
                
#                 # Reset the result and running_length
#                 result = [para]  # Start a new result with the current paragraph
#                 running_length = len(para)  # Set running_length to the current paragraph's length

#         # print(''.join(result))
#         # time.sleep(2)
#         translated_result, runningtokens = GPTranslate_nopop(''.join(result))
#         # print(translated_result)
#         translation.append(translated_result)
        
#         tokensusedline = "Note: This translation used " + str(totaltokens) + " GPT tokens"
#         translation.append(tokensusedline)
#         print(tokensusedline)
        
#         filename = f"{title}.txt"
#         full_path = os.path.join(folder_path, filename)
#         with open(full_path, 'w') as file:
#             file.write(''.join(translation))
#         print("saved to", full_path)
            
#     else:
#         print("Failed to retrieve the webpage. Status code:", response.status_code)